In [26]:
import os
import pandas as pd
os.environ['GRADIENT_WORKSPACE_ID']='9d0447f2-fcd4-4177-9145-9f019fd59f1e_workspace'
os.environ['GRADIENT_ACCESS_TOKEN']='cPErsUMgadGMbzeq8z8W36eJn7UA0Uob'

In [29]:
df1 = pd.read_csv("https://raw.githubusercontent.com/CS-5302/CS-5302-Project-Group-15/main/Datasets/testing/dataset_processed.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/CS-5302/CS-5302-Project-Group-15/main/Datasets/testing/disease_symptom.csv")
df1_pr = df1[["prompts", "results"]]
df2_pr = df2[["prompts", "results"]]
combined_df = pd.concat([df1_pr, df2_pr], ignore_index=True)
combined_df.to_csv("combined_df.csv", index=False)

In [22]:
from gradientai import Gradient


def main():
    gradient = Gradient()

    base_model = gradient.get_base_model(base_model_slug="nous-hermes2")

    new_model_adapter = base_model.create_model_adapter(
        name="meta/llama-2-7b:73001d654114dad81ec65da3b834e2f691af1e1526453189b7bf36fb3f32d0f9"
    )
    print(f"Created model adapter with id {new_model_adapter.id}")


    sample_query = f"###{df['prompts'][0]} ### Response:"
    print(f"Asking: {sample_query}")
    ## Before Finetuning
    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    print(f"Generated(before fine tuning): {completion}")

    samples=[
        {"inputs":f"### Instruction: {df['prompts'][0]} \n\n### Response: {df['results'][0]}"},
        {"inputs":f"### Instruction: {df['prompts'][1]} \n\n### Response: {df['results'][1]}"},
        {"inputs":f"### Instruction: {df['prompts'][2]} \n\n### Response: {df['results'][2]}"},
        {"inputs":f"### Instruction: {df['prompts'][3]} \n\n### Response: {df['results'][3]}"}
        
    ]

    ## Lets define parameters for finetuning
    num_epochs=3
    count=0
    while count<num_epochs:
      print(f"Fine tuning the model with iteration {count + 1}")
      new_model_adapter.fine_tune(samples=samples)
      count=count+1

    #after fine tuning
    completion = new_model_adapter.complete(query=sample_query, max_generated_token_count=100).generated_output
    print(f"Generated(after fine tuning): {completion}")
    new_model_adapter.delete()
    gradient.close()

if __name__ == "__main__":
    main()

Created model adapter with id aadb1644-beac-4cad-8c09-e510eef2d43f_model_adapter
Asking: ###I have Fever, Fatigue, Difficulty Breathing. What disease do i have? ### Response:
Generated(before fine tuning):  Based on the symptoms you have described, it is possible that you may have a respiratory illness such as pneumonia, influenza, or COVID-19. However, it is recommended that you consult with a healthcare professional for a proper diagnosis and treatment.
Fine tuning the model with iteration 1
Fine tuning the model with iteration 2
Fine tuning the model with iteration 3
Generated(after fine tuning):  You have Asthma.
